In [199]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

In [200]:
def extract_game_data(game):
    teams = game.find(class_='h-text-left').get_text()
    score = game.find(class_='h-text-center').get_text()
    odds = game.find_all(class_='table-main__odds')
    for g in game.find_all('a',href=True):
        game_url=g['href']
    bets=[]
    winner=np.nan
    w=-1
    for odd in odds:
        w = w+1
        if odd.has_attr('data-odd'):
            #print(odd.attrs['data-odd'])
            bets.append(odd.attrs['data-odd'])
        else:
            # this is colored cell
            spans = odd.find('span')
            for span in spans:
                #print(span.find('span').attrs['data-odd'])
                bets.append(span.find('span').attrs['data-odd'])
                winner=w
    date = game.find(class_='h-text-right').get_text()

    # split team to home_team and away_team
    t = teams.split(' - ')
    home_team = t[0]
    away_team = t[1]
    try:
        return teams, home_team, away_team, score, bets[0], bets[1], bets[2], date, winner, game_url
    except:
        return teams, home_team, away_team, score, bets, bets, bets, date, winner, game_url


In [201]:
# open the urls forlder and read urls files to get pages to scrap
SRC_FOLDER = '../../src/urls'
FILENAME = 'urls.csv'

TABLE_FOLDER = '../../data/raw/'
RAW_FILE = 'games_2000_2022.csv'


urls = pd.read_csv(f'{SRC_FOLDER}/{FILENAME}')
#urls.columns=['url']
print(urls.shape)
urlslist = urls.urllink.tolist()
urlslist



(1, 1)


['https://www.betexplorer.com/soccer/england/premier-league-2021-2022/results/']

In [202]:
urlslist=[]
for y in range(2021,2022):

    url = f'https://www.betexplorer.com/soccer/england/premier-league-{y}-{y+1}/results/'
    urlslist.append(url)

In [203]:
games = []

# get all table row from the page source
for url in urlslist:
    pos = url.split('/')
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tag_tr = soup.find_all('tr')

# table row interesting me is those with odd-data
    tags=[]
    for tag in tag_tr:
        if 'data-odd' in str(tag):
            tags.append(tag)

# iterate through the list of table row
# each row is agame with 6 fields
    """
    - the game (home, away team)
    - correct score
    - 3 odds (1,N,2)
    - game date
    """

    from inspect import classify_class_attrs


    for tag in tags:
        p=str(tag)
        game =  BeautifulSoup(p, 'html.parser')    
        teams, home_team, away_team, score, odd_1, odd_N, odd_2, date, winner, game_url = extract_game_data(game)

        # COMPUTE DATE
        """
        if month is >= 01 so its year +1 elese it's year
        """
        years = pos[5].split('-')
        ds = date.split('.')
        c_date=date
        if len(date)==6:
            if int(ds[1])>=7:
                c_date = f'{date}{years[2]}'
            else:
                c_date = f'{date}{years[3]}'


        games.append([c_date, home_team, away_team, score, odd_1, odd_N, odd_2, winner,pos[5], game_url])

        # get the game



In [204]:
games=pd.DataFrame(games)
games.columns = ['date','home_team','away_team','score','odd_1','odd_N','odd_2','winner','league_season','game_url']
games

,date,home_team,away_team,score,odd_1,odd_N,odd_2,winner,league_season,game_url
0,22.05.2022,Arsenal,Everton,5:1,1.34,5.50,8.98,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/arsen...
1,22.05.2022,Brentford,Leeds,1:2,2.32,3.79,2.88,2,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/brent...
2,22.05.2022,Brighton,West Ham,3:1,2.66,3.46,2.65,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/brigh...
3,22.05.2022,Burnley,Newcastle,1:2,2.47,3.27,3.01,2,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/burnl...
4,22.05.2022,Chelsea,Watford,2:1,1.19,7.41,15.39,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/chels...
5,22.05.2022,Crystal Palace,Manchester Utd,1:0,2.81,3.47,2.50,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/cryst...
6,22.05.2022,Leicester,Southampton,4:1,1.70,4.26,4.54,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/leice...
7,22.05.2022,Liverpool,Wolves,3:1,1.13,9.19,19.26,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/liver...
8,22.05.2022,Manchester City,Aston Villa,3:2,1.17,8.21,15.57,0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/manch...
9,22.05.2022,Norwich,Tottenham,0:5,9.94,5.55,1.31,2,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/norwi...


In [205]:
games.isnull().sum()

date             0
home_team        0
away_team        0
score            0
odd_1            0
odd_N            0
odd_2            0
winner           0
league_season    0
game_url         0
dtype: int64

In [206]:
games.to_csv(f'{TABLE_FOLDER}{RAW_FILE}')